In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
print(os.environ.get('CUDA_VISIBLE_DEVICES'))

2


In [2]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# CUDA 장치 사용 가능 여부 확인
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask
    


In [10]:
# transform = A.Compose(
#     [   
#         #A.Resize(224, 224),
#         A.Resize(256, 256),
#         A.Normalize(),
#         ToTensorV2()
#     ]
# )

# dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
# valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
# valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

In [11]:
# 데이터셋 클래스 정의와 데이터 로더 설정
transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

train_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM", './train_source.csv'), transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM", './val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)


In [12]:

import torch
import torch.nn as nn
from torchvision import models

class UNetResNet(nn.Module):
    def __init__(self, num_classes):
        super(UNetResNet, self).__init__()
        # 인코더: ResNet
        self.encoder = models.resnet18(pretrained=True)
        # 디코더: Upsampling 레이어 추가
        self.decoder1 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)
        self.decoder2 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.decoder3 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.decoder4 = nn.ConvTranspose2d(64, num_classes, kernel_size=4, stride=2, padding=1)
    
    def forward(self, x):
        x = self.encoder.conv1(x)
        x = self.encoder.bn1(x)
        x = self.encoder.relu(x)
        x = self.encoder.maxpool(x)
        # 인코더의 layer1, layer2, layer3, layer4를 순차적으로 실행
        x1 = self.encoder.layer1(x)
        x2 = self.encoder.layer2(x1)
        x3 = self.encoder.layer3(x2)
        x4 = self.encoder.layer4(x3)
        # 디코더 부분
        x = self.decoder1(x4)
        x = self.decoder2(x + x3)
        x = self.decoder3(x + x2)
        x = self.decoder4(x + x1)
        return x

# 모델 초기화
model = UNetResNet(num_classes=13)  # 마스크 클래스 수에 맞춰 설정


model.to(device)



UNetResNet(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [13]:
import wandb
import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="Design_PJ",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.005,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 15,
#     }
# )

# 클래스(레이블) 수
num_classes = 13
batch_size=32

# loss function과 optimizer 정의
#criterion = torch.nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss(ignore_index=12)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou


# training loop
for epoch in range(10):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    class_ious = []
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(train_dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # validation 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            class_ious.append(iou)
            train_class_ious.append(iou)

    # mIoU 계산
    train_mIoU = np.mean(class_ious)
    train_class_ious = np.array(train_class_ious).reshape(num_classes, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    #mIoU = np.mean(class_ious)
    #print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(dataloader)}, train mIoU Score: {mIoU:.4f}")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

    # validation
    val_loss = 0
    class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)
            val_class_ious = []
            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                class_ious.append(iou)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(num_classes, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

                
    # for class_id, iou in enumerate(class_ious):
    # print(f'Class {class_id} IoU: {iou:.4f}')           
   
    # mIoU 계산
    val_mIoU = np.mean(class_ious)

#     # log metrics to wandb
#     wandb.log({"train_loss": (epoch_loss/len(dataloader))})
#     wandb.log({"Train Score": train_mIoU})
#     wandb.log({"val_loss": (val_loss/len(valid_dataloader))})
#     wandb.log({"Validation Score": val_mIoU})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

  0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/69 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
for images, masks in train_dataloader:
    print(f"Image shape: {images.shape}, Mask shape: {masks.shape}")
    break


Image shape: torch.Size([32, 3, 224, 224]), Mask shape: torch.Size([32, 224, 224])


In [5]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


# Define the UNetResNet model
class UNetResNet(nn.Module):
    def __init__(self, num_classes):
        super(UNetResNet, self).__init__()
        self.encoder = models.resnet18(pretrained=True)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, num_classes, kernel_size=4, stride=2, padding=1)
        )
        self.final_layer = nn.Softmax(dim=1)

    def forward(self, x):
        resnet_features = self.encoder(x)
        x = self.decoder(resnet_features)
        x = self.final_layer(x)
        return x

# Data transformations
transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

# Define the data loaders
train_dataset = CustomDataset(csv_file='/mnt/nas27/Dataset/Samsung_DM/train_source.csv', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

valid_dataset = CustomDataset(csv_file='/mnt/nas27/Dataset/Samsung_DM/val_source.csv', transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize the model
num_classes = 13  # Modify this based on your task
model = UNetResNet(num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs=10
# Training loop (you can adjust this as needed)
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        images, masks = batch['image'], batch['mask']
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        for batch in valid_dataloader:
            images, masks = batch['image'].to(device), batch['mask'].to(device)
            outputs = model(images)
            # Calculate and print validation metrics as needed


TypeError: list indices must be integers or slices, not str